<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg">
## Open Machine Learning Course
<center>Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/), Data Scientist at Mail.ru Group <br>
    All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.
You may use this material for any purpose (you can edit, correct and use it as example) exept commercial use with mandatory citation of author.

# <center> Assignment #6 (demo).
## <center>  Exploring OLS, Lasso and Random Forest in a regression task
    
<img src=https://habrastorage.org/webt/-h/ns/aa/-hnsaaifymavmmudwip9imcmk58.jpeg width=30%>

**Fill in the missing code and choose answers in [this](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit) web form.**

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**We are working with UCI Wine quality dataset (no need to download it – it's already there, in course repo and in Kaggle Dataset).**

In [2]:
data = pd.read_csv('../input/winequality-white.csv')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [5]:
data['quality'].value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [6]:
X = data.drop(columns=['quality'], axis=1)
y = data['quality']

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [7]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

y_pred_train = linreg.predict(X_train_scaled)
y_pred_holdout = linreg.predict(X_holdout_scaled)

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [8]:
print(f"R^2: {linreg.score(X_holdout_scaled, y_holdout)}")

print(f"Mean squared error (train): {mean_squared_error(y_train, y_pred_train):.3f}")
print(f"Mean squared error (test): {mean_squared_error(y_holdout, y_pred_holdout):.3f}")
      
print(f"Root mean squared error (train): {np.sqrt(mean_squared_error(y_train, y_pred_train)):.3f}")
print(f"Root mean squared error (test): {np.sqrt(mean_squared_error(y_holdout, y_pred_holdout)):.3f}")

R^2: 0.24987105940272702
Mean squared error (train): 0.558
Mean squared error (test): 0.584
Root mean squared error (train): 0.747
Root mean squared error (test): 0.764


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [9]:
linreg_coef = pd.DataFrame(list(zip(data.columns, np.abs(linreg.coef_))))
linreg_coef.columns = ['feature', 'coef']

linreg_coef = linreg_coef.sort_values(by='coef', ascending=False)
linreg_coef

,feature,coef
7,density,0.665720
3,residual sugar,0.538164
1,volatile acidity,0.192260
8,pH,0.150036
10,alcohol,0.129533
0,fixed acidity,0.097822
9,sulphates,0.062053
5,free sulfur dioxide,0.042180
6,total sulfur dioxide,0.014304
4,chlorides,0.008127


## Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [10]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

y_pred_train_lasso = lasso1.predict(X_train_scaled)
y_pred_holdout_lasso = lasso1.predict(X_holdout_scaled)

In [11]:
print(f"Lasso R^2: {lasso1.score(X_holdout_scaled, y_holdout)}")

print(f"Lasso Mean squared error (train): {mean_squared_error(y_train, y_pred_train_lasso):.3f}")
print(f"Lasso Mean squared error (test): {mean_squared_error(y_holdout, y_pred_holdout_lasso):.3f}")
      
print(f"Lasso Root mean squared error (train): {np.sqrt(mean_squared_error(y_train, y_pred_train_lasso)):.3f}")
print(f"Lasso Root mean squared error (test): {np.sqrt(mean_squared_error(y_holdout, y_pred_holdout_lasso)):.3f}")

Lasso R^2: 0.2634608744708563
Lasso Mean squared error (train): 0.564
Lasso Mean squared error (test): 0.574
Lasso Root mean squared error (train): 0.751
Lasso Root mean squared error (test): 0.757


**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [12]:
lasso1_coef = pd.DataFrame(list(zip(data.columns, np.abs(lasso1.coef_))))
lasso1_coef.columns = ['feature', 'coef']

lasso1_coef = lasso1_coef.sort_values(by='coef', ascending=True)
lasso1_coef

,feature,coef
0,fixed acidity,0.000000
2,citric acid,0.000000
6,total sulfur dioxide,0.000000
4,chlorides,0.002747
9,sulphates,0.029722
5,free sulfur dioxide,0.043088
8,pH,0.067277
1,volatile acidity,0.188479
7,density,0.235492
3,residual sugar,0.256363


**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [13]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv='warn', eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [14]:
lasso_cv.alpha_

0.0003739937302478798

**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [15]:
lasso_cv_coef = pd.DataFrame(list(zip(data.columns, np.abs(lasso_cv.coef_))))
lasso_cv_coef.columns = ['feature', 'coef']

lasso_cv_coef = lasso_cv_coef.sort_values(by='coef', ascending=True)
lasso_cv_coef

,feature,coef
2,citric acid,0.000000
4,chlorides,0.006558
6,total sulfur dioxide,0.012546
5,free sulfur dioxide,0.042865
9,sulphates,0.060585
0,fixed acidity,0.091858
10,alcohol,0.139555
8,pH,0.145426
1,volatile acidity,0.191992
3,residual sugar,0.523266


**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [16]:
lasso2 = Lasso(alpha=lasso_cv.alpha_, random_state=17)
lasso2.fit(X_train_scaled, y_train)

y_pred_train_lasso = lasso2.predict(X_train_scaled)
y_pred_holdout_lasso = lasso2.predict(X_holdout_scaled)

In [17]:
print(f"Lasso R^2: {lasso2.score(X_holdout_scaled, y_holdout)}")

print(f"Lasso Mean squared error (train): {mean_squared_error(y_train, y_pred_train_lasso):.3f}")
print(f"Lasso Mean squared error (test): {mean_squared_error(y_holdout, y_pred_holdout_lasso):.3f}")
      
print(f"Lasso Root mean squared error (train): {np.sqrt(mean_squared_error(y_train, y_pred_train_lasso)):.3f}")
print(f"Lasso Root mean squared error (test): {np.sqrt(mean_squared_error(y_holdout, y_pred_holdout_lasso)):.3f}")

Lasso R^2: 0.25146874199174585
Lasso Mean squared error (train): 0.558
Lasso Mean squared error (test): 0.583
Lasso Root mean squared error (train): 0.747
Lasso Root mean squared error (test): 0.764


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [18]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

cv_score = np.abs(np.mean(cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error')))
y_pred_train_forest = forest.predict(X_train_scaled)
y_pred_holdout_forest = forest.predict(X_holdout_scaled)

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [19]:
print(f"Random Forest R^2: {forest.score(X_holdout_scaled, y_holdout)}")

print(f"Random Forest Mean squared error (train): {mean_squared_error(y_train, y_pred_train_forest):.3f}")
print(f"Random Forest Mean squared error (cv): {cv_score:.3f}")
print(f"Random Forest Mean squared error (test): {mean_squared_error(y_holdout, y_pred_holdout_forest):.3f}")

Random Forest R^2: 0.4582191552924345
Random Forest Mean squared error (train): 0.075
Random Forest Mean squared error (cv): 0.460
Random Forest Mean squared error (test): 0.422


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [20]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
locally_best_forest.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 630 candidates, totalling 1890 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   42.0s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1890 out of 1890 | elapsed:  1.5min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [21]:
locally_best_forest.best_params_, np.abs(locally_best_forest.best_score_)

({'max_depth': 22, 'max_features': 7, 'min_samples_leaf': 1},
 0.45285564564445485)

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [22]:
forest = RandomForestRegressor(max_depth=22, max_features=7, min_samples_leaf=1, random_state=17)
forest.fit(X_train_scaled, y_train)

cv_score = np.abs(np.mean(cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error')))
y_pred_holdout_forest = forest.predict(X_holdout_scaled)

In [23]:
print(f"Random Forest R^2: {forest.score(X_holdout_scaled, y_holdout)}")

print(f"Random Forest Mean squared error (cv): {cv_score:.3f}")
print(f"Random Forest Mean squared error (test): {mean_squared_error(y_holdout, y_pred_holdout_forest):.3f}")

Random Forest R^2: 0.46402969389833326
Random Forest Mean squared error (cv): 0.453
Random Forest Mean squared error (test): 0.417


**Output RF's feature importance. Again, it's nice to present it as a DataFrame.**<br>
**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [24]:
rf_importance = pd.DataFrame(list(zip(data.columns, forest.feature_importances_)))
rf_importance.columns = ['feature', 'importance']

rf_importance = rf_importance.sort_values(by='importance', ascending=False)
rf_importance

,feature,importance
10,alcohol,0.226548
1,volatile acidity,0.121010
5,free sulfur dioxide,0.109449
6,total sulfur dioxide,0.073495
8,pH,0.073450
4,chlorides,0.073096
7,density,0.072642
3,residual sugar,0.070051
0,fixed acidity,0.061932
2,citric acid,0.059728


**Make conclusions about the performance of the explored 3 models in this particular prediction task.**